In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (195 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'),sep=',',header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales")

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
grouped_data =df.filter('bedrooms=4')
grouped_data.groupBy('date_built').agg({'price':'avg'}).show()

+----------+------------------+
|date_built|        avg(price)|
+----------+------------------+
|      2016|296050.24326347304|
|      2012| 298233.4150805271|
|      2017| 296576.6934782609|
|      2014| 299073.8858447489|
|      2013|299999.38822652755|
|      2011|302141.89581749047|
|      2015|307908.86020761245|
|      2010| 296800.7544776119|
+----------+------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
grouped_data =df.filter('bedrooms=3 and bathrooms=3')
grouped_data.groupBy('date_built').agg({'price':'avg'}).show()

+----------+------------------+
|date_built|        avg(price)|
+----------+------------------+
|      2016|  290555.073964497|
|      2012| 293683.1872074883|
|      2017| 292676.7887740029|
|      2014| 290852.2661870504|
|      2013|295962.27145085804|
|      2011|291117.46706586826|
|      2015| 288770.2966101695|
|      2010|  292859.615942029|
+----------+------------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

grouped_data =df.filter('bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>=2000 ')
grouped_data.groupBy('date_built').agg({'price':'avg'}).show()

+----------+------------------+
|date_built|        avg(price)|
+----------+------------------+
|      2016| 293965.1046511628|
|      2012|307539.97402597405|
|      2017|280317.57692307694|
|      2014| 298264.7183908046|
|      2013|      303676.79375|
|      2011| 276553.8128654971|
|      2015| 297609.9679144385|
|      2010| 285010.2215909091|
+----------+------------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
SELECT view,AVG(price)
FROM home_sales
GROUP BY view
HAVING AVG(price)>=350000


""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|        avg(price)|
+----+------------------+
|  51| 788128.2083333334|
|  54| 798684.8235294118|
|  69| 750537.9411764706|
|  87|         1072285.2|
|  73| 752861.1764705882|
|  64| 767036.6666666666|
|  59|          791453.0|
|  85|1056336.7435897435|
|  52| 733780.2608695652|
|  71| 775651.0952380953|
|  98|1053739.3333333333|
|  99|1061201.4193548388|
|  96|1017815.9166666666|
| 100|         1026669.5|
|  70| 695865.5769230769|
|  61| 746877.5909090909|
|  75|1114042.9428571428|
|  78|1080649.3666666667|
|  89|        1107839.15|
|  77|        1076205.56|
+----+------------------+
only showing top 20 rows

--- 1.388723611831665 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""
SELECT view,AVG(price)
FROM home_sales
GROUP BY view
HAVING AVG(price)>=350000


""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|        avg(price)|
+----+------------------+
|  51| 788128.2083333334|
|  54| 798684.8235294118|
|  69| 750537.9411764706|
|  87|         1072285.2|
|  73| 752861.1764705882|
|  64| 767036.6666666666|
|  59|          791453.0|
|  85|1056336.7435897435|
|  52| 733780.2608695652|
|  71| 775651.0952380953|
|  98|1053739.3333333333|
|  99|1061201.4193548388|
|  96|1017815.9166666666|
| 100|         1026669.5|
|  70| 695865.5769230769|
|  61| 746877.5909090909|
|  75|1114042.9428571428|
|  78|1080649.3666666667|
|  89|        1107839.15|
|  77|        1076205.56|
+----+------------------+
only showing top 20 rows

--- 0.5501861572265625 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [13]:
# 11. Read the parquet formatted data.
partioned_parquet_file=spark.read.parquet('home_sales_partitioned')

In [14]:
# 12. Create a temporary table for the parquet data.
partioned_parquet_file.createOrReplaceTempView("partioned_parquet_temp_file")

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""
SELECT view,AVG(price)
FROM home_sales
GROUP BY view
HAVING AVG(price)>=350000


""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|        avg(price)|
+----+------------------+
|  51| 788128.2083333334|
|  54| 798684.8235294118|
|  69| 750537.9411764706|
|  87|         1072285.2|
|  73| 752861.1764705882|
|  64| 767036.6666666666|
|  59|          791453.0|
|  85|1056336.7435897435|
|  52| 733780.2608695652|
|  71| 775651.0952380953|
|  98|1053739.3333333333|
|  99|1061201.4193548388|
|  96|1017815.9166666666|
| 100|         1026669.5|
|  70| 695865.5769230769|
|  61| 746877.5909090909|
|  75|1114042.9428571428|
|  78|1080649.3666666667|
|  89|        1107839.15|
|  77|        1076205.56|
+----+------------------+
only showing top 20 rows

--- 0.5710544586181641 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False